**Modèle Pré entrainé BERT**

Nous avons utlisé le transfert learning dans ce notebook pour fine-tuner BERT afin d'entrainer un réseau de neurones pour classifier les questions posées en entrée dans les différentes catégories.

In [ ]:
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from  sklearn import preprocessing
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


**Installation de Bert Text**

In [ ]:
!pip3 install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.4 MB/s 


**Importation des bibliothèques BERT**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#récupértion du dataset
with open('/content/drive/Shareddrives/Projets ING3 IA 2021-2022/Natural Language Processing/order.json') as json_data:
    data_dict = json.load(json_data)

Après avoir récupérer le dataset sous format json, on extrait dans un dataFrame les questions et leurs tags correspondants (label)

In [ ]:
#retourne le dataframe et la liste des categorie de nos questions pour entrainer notre reseau
def create_df_from_dic(data_dict):
  df = pd.DataFrame(columns=['question', 'label'])
  data = {}
  classes = []
  for conv in data_dict['intents']:
    data[conv['tag']] = conv['patterns']

  for key, value in data.items():
    classes.append(key)
    for question in value:
      df = df.append({'question':question, 'label':key}, ignore_index=True)
  return (df,classes)


In [ ]:
df = create_df_from_dic(data_dict)[0]

'can you sing'

In [ ]:
classes = create_df_from_dic(data_dict)[1]
classes

['greeting',
 'goodbye',
 'thanks',
 'talk',
 'alive',
 'chatbot',
 'about me',
 'creator',
 'myself',
 'God',
 'joke',
 'killing',
 'bookings',
 'stories',
 'weather',
 'google',
 'wikipedia',
 'news',
 'noanswer',
 'options',
 'order_tracking',
 'order_components',
 'missing_id',
 'Location',
 'search_person_by_id',
 'appointment status',
 'check_leave',
 'cost_lowering',
 'forgot_password',
 'email_id',
 'manufacturing_problems',
 'search_department',
 'competitors_in_market',
 'key_customers',
 'supplier_info',
 'highest_grossing',
 'connect_people',
 'project_handling_queries',
 'solve_problems',
 'version_update',
 'HR_related_problem',
 'factors_impacting_sale',
 'predict_performance',
 'customer_satisfaction',
 'maintainence',
 'gadgets',
 'commission',
 'invalid',
 'noans',
 'turnover',
 'predict_delay',
 'name',
 'about',
 'configuration',
 'Weather',
 'leave',
 'hours',
 'cabin',
 'domain']

**On utilise LabelEncoder de scikit learn pour numériser notre colone "label"**

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df['label'])
label_encoded = le.transform(df['label'])

**Ensuite on utilise un oneHot encoding pour la colonne label**

In [ ]:
def one_hot_encoding(num_class):
  res = [0]*58 
  res[num_class] = 1
  return res

In [ ]:
df_final = df
df_final['label_encoded'] = label_encoded
df_final['encoded_class'] = df_final['label_encoded'].apply(lambda x:one_hot_encoding(x))
df_final

,question,label,label_encoded,encoded_class
0,Hi there,greeting,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,How are you,greeting,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Is anyone there?,greeting,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Hey,greeting,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Hola,greeting,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
174,Where is Project head department?,cabin,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
175,where is cafeteria,cabin,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
176,guide me to the canteen,cabin,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
177,How to improve team members domain knowledge,domain,19,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**Chargement des modèles BERT ( On utilise BERT à 12 couche d'encoding qui est plus léger)**

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Fonction qui permet d'encoder les questions
Elle retourne pour chacune des question un vecteur 768 

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
x_train = get_sentence_embeding(list(df_final['question']))
y_train = df_final['encoded_class']

In [ ]:
x_train

<tf.Tensor: shape=(179, 768), dtype=float32, numpy=
array([[-0.93085593, -0.39940786, -0.7998353 , ..., -0.44739163,
        -0.7021608 ,  0.9310605 ],
       [-0.9322528 , -0.29204777, -0.4289326 , ..., -0.21647911,
        -0.6645844 ,  0.9445238 ],
       [-0.94923663, -0.36109567, -0.88410205, ..., -0.55859864,
        -0.7125871 ,  0.94880736],
       ...,
       [-0.8959174 , -0.17651793, -0.4016    , ..., -0.09311155,
        -0.6670421 ,  0.926026  ],
       [-0.9057771 , -0.43960556, -0.8737796 , ..., -0.7436275 ,
        -0.68930686,  0.82095516],
       [-0.9103897 , -0.42661947, -0.867451  , ..., -0.79927534,
        -0.6423335 ,  0.7456383 ]], dtype=float32)>

In [ ]:
y_train

0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
174    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
175    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
176    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
177    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
178    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: encoded_class, Length: 179, dtype: object

In [ ]:
y_train = y_train.values.tolist()

**On affiche le cosine similarity entre quelques entrées x_train pour vérifier que l'encodage par BERT forunit de bons résultats**

In [ ]:
#similarité entre 'hi there' et 'how are you'
cosine_similarity( [x_train[0]] , [x_train[0]] )[0][0] 

0.9999999

In [ ]:
#similarité entre 'hi there' et 'can you sing'
cosine_similarity( [x_train[0]] , [x_train[17]] )[0][0] 

0.66603696

**Implémentation du réseau de neurones avec Keras**

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))
model.summary()

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               98432     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 58)                3770      
                                                                 
Total params: 110,458
Trainable params: 110,458
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


36/36 [==============================] - 1s 4ms/step - loss: 4.3542 - accuracy: 0.0168
Epoch 2/200
36/36 [==============================] - 0s 4ms/step - loss: 4.0557 - accuracy: 0.0391
Epoch 3/200
36/36 [==============================] - 0s 4ms/step - loss: 4.0282 - accuracy: 0.0447
Epoch 4/200
36/36 [==============================] - 0s 4ms/step - loss: 4.0086 - accuracy: 0.0279
Epoch 5/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9935 - accuracy: 0.0559
Epoch 6/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9812 - accuracy: 0.0559
Epoch 7/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9821 - accuracy: 0.0447
Epoch 8/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9652 - accuracy: 0.0447
Epoch 9/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9573 - accuracy: 0.0391
Epoch 10/200
36/36 [==============================] - 0s 4ms/step - loss: 3.9458 - accuracy: 0.0391
Epoch 11/200
36/36 [=

!!!!
LA METHODE FONCTIONNE BIEN. LE PROBLEME QUI NOUS EMPECHE D'AVOIR DE MEILLEURS RESULTATS EST LA TAILLE DE NOTRE BASE DE DONNEES QUI N'EST PAS CONSEQUENTE 